In [1]:
###VacationPy

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps.datasets
import gmaps
import time
import os

# Import API key
from config import g_key

In [3]:
### Store Part I results into DataFrame
city_data_path = "Resources/clean_city_data.csv"

# Read the  data
cities_df = pd.read_csv(city_data_path)
cities_df.head(5)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,"ca mau,vn",9.1769,105.1500,88.38,59,90,12.19,VN,1626923432
1,"ayagoz,kz",47.9714,80.4392,71.44,51,100,19.44,KZ,1626923683
2,"dano,bf",11.1500,-3.0667,76.06,74,100,8.08,BF,1626923684
3,"novyy urengoy,ru",66.0833,76.6333,59.77,87,100,16.87,RU,1626923682
4,"black river,jm",18.0264,-77.8487,83.98,72,80,4.50,JM,1626923684


In [4]:
###Humidity Heatmap

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Use the Lat and Lng as locations and Humidity as the weight
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities_df[["Lat", "Lng"]]

# Add Heatmap layer to map & Create a heat map that displays the humidity for every city from the part I of the homework.
fig = gmaps.figure()
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
heat_map=fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=2.5))
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Create new DataFrame fitting weather criteria & narrow down the cities to fit weather conditions.
narrowed_cities_df = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) & (cities_df["Cloudiness"] == 0), :]
# Drop any rows will null values.
narrowed_cities_df = narrowed_cities_df.dropna(how='any') 
narrowed_cities_df.reset_index(inplace=True)
del narrowed_cities_df['index']
narrowed_cities_df.head(5)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,"mahon,es",39.8885,4.2658,75.76,88,0,3.44,ES,1626923687
1,"pop,uz",40.8736,71.1089,79.29,32,0,3.60,UZ,1626923701
2,"yulara,au",-25.2406,130.9889,75.04,29,0,20.71,AU,1626923737
3,"misratah,ly",32.3754,15.0925,77.68,71,0,5.70,LY,1626923688
4,"cordoba,es",38.0000,-4.8333,73.13,40,0,2.30,ES,1626923744


In [8]:
###Hotel Map

In [9]:
# Store into variable named hotel_df
hotel_df = narrowed_cities_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,"mahon,es",ES,39.8885,4.2658,
1,"pop,uz",UZ,40.8736,71.1089,
2,"yulara,au",AU,-25.2406,130.9889,
3,"misratah,ly",LY,32.3754,15.0925,
4,"cordoba,es",ES,38.0000,-4.8333,


In [10]:
#URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for hotels with 5000 meters.
params = {
        "radius": 5000,
        "types" : "hotel",
        "keyword": "hotel",    
        "key": g_key
    }
# Iterate through: Use the lat/lng to identify hotels 
for index, row in hotel_df.iterrows():
    # not referencing row['Lat']; 
    # only putting ['Lat'] which is a list with the string 'Lat' as an item
    # same for lng and city
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    #change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    #make request and print url
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result--- skipping")
        print("-------------------------")
        #delay the code by one second to avoid an error
    time.sleep(1)
    print("--------------------End of Search---------------")

Retrieving Results for Index 0: mahon,es.
Closest hotel in mahon,es is Jardí de ses Bruixes Boutique Hotel.
--------------------End of Search---------------
Retrieving Results for Index 1: pop,uz.
Closest hotel in pop,uz is Shox Hotel.
--------------------End of Search---------------
Retrieving Results for Index 2: yulara,au.
Closest hotel in yulara,au is Sails in the Desert.
--------------------End of Search---------------
Retrieving Results for Index 3: misratah,ly.
Closest hotel in misratah,ly is فندق الماسة.
--------------------End of Search---------------
Retrieving Results for Index 4: cordoba,es.
Missing field/result--- skipping
-------------------------
--------------------End of Search---------------
Retrieving Results for Index 5: bardiyah,ly.
Closest hotel in bardiyah,ly is Hotel Burdy.
--------------------End of Search---------------
Retrieving Results for Index 6: bin qirdan,tn.
Closest hotel in bin qirdan,tn is hotel rayes.
--------------------End of Search---------------

In [11]:
# Store the first Hotel result into the DataFrame.
hotel_df.head(1)

,City,Country,Lat,Lng,Hotel Name
0,"mahon,es",ES,39.8885,4.2658,Jardí de ses Bruixes Boutique Hotel


In [12]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer and info box content ontop of heat map

# Use the Lat and Lng as locations and Humidity as the weight
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.9)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))